In [28]:
import numpy as np
from PIL import Image
from IPython.display import display

def matrix_as_image(matrix):
    matrix = np.clip(matrix, 0, 255).astype(np.uint8)

    
    if len(matrix.shape) == 2:
        img = Image.fromarray(matrix, mode='L')
    elif len(matrix.shape) == 3 and matrix.shape[2] == 3:
        img = Image.fromarray(matrix, mode='RGB')
    else:
        raise ValueError("Matrix must be 2D (grayscale) or 3D (RGB).")
    return img


def load_image_as_matrix(image_path):
    image = Image.open(image_path)
    return np.array(image)

def save_matrix_as_image(matrix, output_path):
    matrix = np.clip(matrix, 0, 255).astype(np.uint8)
    img = Image.fromarray(matrix)
    img.save(output_path)

def encrypt_block(matrix_block, key_matrix):
    return np.dot(key_matrix, matrix_block)

def decrypt_block(encrypted_block, key_matrix_inv):
    return np.dot(key_matrix_inv, encrypted_block)

def generate_key_matrix(size):
    while True:
        key_matrix = np.random.randint(1, 10, (size, size))
        try:
            key_matrix_inv = np.linalg.inv(key_matrix)
            return key_matrix, key_matrix_inv
        except np.linalg.LinAlgError:
            continue

if __name__ == "__main__":
    image_path = r"pexels-johannes-havn-3218340.jpg"
    output_encrypted_path = "encrypted_image.jpg"
    output_decrypted_path = "decrypted_image.jpg"
    image_matrix = load_image_as_matrix(image_path)

    key_size = 3 
    channels = image_matrix.shape[2]
    
    padded_rows = key_size - (image_matrix.shape[0] % key_size) if image_matrix.shape[0] % key_size != 0 else 0
    padded_cols = key_size - (image_matrix.shape[1] % key_size) if image_matrix.shape[1] % key_size != 0 else 0

    
    padded_image_matrix = np.pad(
        image_matrix, 
        ((0, padded_rows), (0, padded_cols), (0, 0)),
        mode='constant'
    )

    num_blocks_row = padded_image_matrix.shape[0] // key_size
    num_blocks_col = padded_image_matrix.shape[1] // key_size

    key_matrix, key_matrix_inv = generate_key_matrix(key_size)

    encrypted_matrix = np.zeros_like(padded_image_matrix, dtype=np.float64)

    
    for channel in range(channels):
        for i in range(num_blocks_row):
            for j in range(num_blocks_col):
                block = padded_image_matrix[
                    i * key_size:(i + 1) * key_size, 
                    j * key_size:(j + 1) * key_size,
                    channel
                ]
                encrypted_matrix[
                    i * key_size:(i + 1) * key_size, 
                    j * key_size:(j + 1) * key_size,
                    channel
                ] = encrypt_block(block, key_matrix)

    save_matrix_as_image(encrypted_matrix, output_encrypted_path)

    decrypted_matrix = np.zeros_like(encrypted_matrix, dtype=np.float64)

    for channel in range(channels):
        for i in range(num_blocks_row):
            for j in range(num_blocks_col):
                block = encrypted_matrix[
                    i * key_size:(i + 1) * key_size, 
                    j * key_size:(j + 1) * key_size,
                    channel
                ]
                decrypted_matrix[
                    i * key_size:(i + 1) * key_size, 
                    j * key_size:(j + 1) * key_size,
                    channel
                ] = decrypt_block(block, key_matrix_inv)

    decrypted_matrix = decrypted_matrix[:image_matrix.shape[0], :image_matrix.shape[1], :]

    save_matrix_as_image(decrypted_matrix, output_decrypted_path)
